This is an end to end example of how to train a transformer on a process, probe it's internal activations for the MSP structure, and visualize it

Installing all the dependencies into the .venv

Now we set up all of the necessary configs for training a model #TODO: Put some explanation of all of the configs

In [ ]:
import pathlib

from epsilon_transformers.training.configs import TrainConfig, RawModelConfig, OptimizerConfig, ProcessDatasetConfig, PersistanceConfig, LoggingConfig
from epsilon_transformers.training.train import train_model


model_config = RawModelConfig(
            d_vocab=2,
            d_model=100,
            n_ctx=10,
            d_head=48,
            n_head=12,
            d_mlp=12,
            n_layers=2,
        )

optimizer_config = OptimizerConfig(
    optimizer_type='adam',
    learning_rate=1.06e-4,
    weight_decay=0.8
)

dataset_config = ProcessDatasetConfig(
    process='z1r',
    batch_size=5,
    num_tokens=50000,
    test_split=0.15
)

persistance_config = PersistanceConfig(
    location='local',
    checkpoint_dir=pathlib.Path('/root/temp_models'), # Temporary Hack. Don't run this in local
    checkpoint_every_n_tokens=100
)

mock_config = TrainConfig(
    model=model_config,
    optimizer=optimizer_config,
    dataset=dataset_config,
    persistance=persistance_config,
    logging=LoggingConfig(project_name="testing-logging-output", wandb=True),
    verbose=True,
    seed=1337
)
model, metrics = train_model(mock_config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: melembroucarlitos44. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


now that we have the model, we'll run the analysis on it

In [ ]:
from epsilon_transformers.process.processes import ZeroOneR
from epsilon_transformers.analysis.activation_analysis import find_msp_subspace_in_residual_stream

process = ZeroOneR()

ground_truth_belief_states_reshaped, predicted_beliefs = find_msp_subspace_in_residual_stream(model=model, process=process, num_sequences=1000)


And now that we have the subspace, we should go ahead and visualize it

In [ ]:
from epsilon_transformers.visualization.plots import generate_belief_state_figures_datashader


fig = generate_belief_state_figures_datashader(ground_truth_tensor=ground_truth_belief_states_reshaped, predicted_beliefs=predicted_beliefs, plot_triangles=True)
fig.show()